In [ ]:
import tifffile
import numpy as np
import os
from collections import defaultdict
import sys
import re
import h5py

In [ ]:
name_regex = re.compile(r"""[\w-]+Pos_(\d+)_(\d+)""")
image_directory = 'Sm_Movies/'
name = image_directory

In [ ]:
#def get_all_tif_paths(image_directory):
paths = defaultdict(set)
for directory, subdirs, filenames in os.walk(image_directory):
    if not filenames:
        continue
    for filename in filenames:
        if not filename.endswith('ome.tif'):
            continue
        paths[directory].add(os.path.join(directory, filename))

In [ ]:
tif_stack = []
for directory, tifs in paths.items():
    tif_stack += list(tifs)
print tif_stack

In [ ]:
#def get_tif_positions(tif_stack):
tif_axes = {}
best_first = 0
best_second = 0
for file_path in tif_stack:
    filename = os.path.split(file_path)[1]
    axis_positions = name_regex.search(filename)
    first = int(axis_positions.group(1))
    second = int(axis_positions.group(2))
    best_first = max(first, best_first)
    best_second = max(second, best_second)
    tif_axes[file_path] = (first, second)
if best_second > best_first:
    # the second thing is the major axis, so we need to invert them
    self._axes = {file_path: (second, first) for file_path, (first, second) in tif_axes.items()}
print tif_axes

In [ ]:
#def create_h5_dictionary(positions):
h5_entries = defaultdict(lambda: defaultdict(int))
for file_name, position in tif_axes.items():
    if 'Cy3' in file_name:
        major_position = str(position[0])
        minor_position = str(position[1])
        positions = '(Major, minor) = ({}, {})'.format(major_position, minor_position)
        #print positions
        with tifffile.TiffFile(file_name) as tif:
            img = tif.asarray()
            sum_img = np.sum(img, dtype=np.int, axis=0)
            #print sum_img.size, sum_img.shape
            h5_entries['Cy3'][positions] = sum_img
    if 'Cy5' in file_name:
        #print file_name
        major_position = str(position[0])
        minor_position = str(position[1])
        positions = '(Major, minor) = ({}, {})'.format(major_position, minor_position)
        #print positions
        with tifffile.TiffFile(file_name) as tif:
            image = tif.asarray()
            #for adjustment in self._adjustments:
            #    image = adjustment(image)
            sum_image = np.sum(image, dtype=np.int, axis=0)
            #print sum_image.size, sum_image.shape
            h5_entries['Cy5'][positions] = sum_image
    #return h5_entries
    print h5_entries.keys()

In [ ]:
#def convert_image_dict_to_h5(name, datadict):
#    """Convert nested dictionary to h5 file
#   
#    Parameters
#    ----------
#    name: str
#        Name of the movie; used to name h5 file
#    
#    datadict: dict
#        Dictionary mapping channel names and positions to pixel intensity arrays
#
#    """
    
#h = h5py.File('{}.h5'.format(name), 'w')
for ch_name in h5_entries.keys():
    for pos, image in h5_entries[ch_name].items():
        position_regex = re.findall('\d+', pos)
        major_position = position_regex[0]
        h_name = 'Position_' + major_position + '.h5'
        if os.path.exists(h_name):
            print "Working with existing file: " + h_name
            h = h5py.File('Position_%s.h5' % major_position, 'r+')
            #print 'Opening: ' + str(h.name) 
            if ch_name not in h:
                group = h.create_group(ch_name)
            else:
                group = h[ch_name]
            if pos not in h:
                dataset = group.create_dataset(pos, image.shape, dtype = image.dtype) 
            else:
                datset = group[pos]
            dataset[...] = image
            h.close()
        else:
            print "Creating New File: " + h_name
            h = h5py.File('Position_{}.h5'.format(major_position), 'w')
            #print "Creating HDF5 File: " + str(h)
            if ch_name not in h:
                group = h.create_group(ch_name)
                #print "Creating Channel: " + ch_name
            else:
                group = h[ch_name]
                #print "Using Channel: " + ch_name

            if pos not in h:
                dataset = group.create_dataset(pos, image.shape, dtype = image.dtype) 
                #print "Dataset = " + str(dataset.name)
                dataset[...] = image
            else:
                datset = group[pos]
                #print "Opening Dataset= " + stre(dataset.name)
            dataset[...] = image
            h.close()            
#         #print h.name